 # Jupyter part code summary 
     This part of code is the code summary existing on jupyter notebook. It mainly includes video input and output, core target detection algorithm and UART serial communication. The actual operation requires one (two for test) HDMI line, one display, one computer, one camera, one board, one boost chip, special track, motor and its driver. 
                                                                                                                 ------fan 2019.11.23

In [ ]:
#测试1：简单的输入连接输出
# 首先加载基本的overlay，加载视频处理IP 加载PL以启用按键监测 调用时间模块计算fps 实例化HDMI输入输出模组
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from pynq import PL
import time

print ('1') #6s
base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

print ('2') #

# 设置输入输出格式（hdmi_in设置为默认，hdmi_out设置为和in相同的格式）
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)

# 调用函数，开始hdmi输入输出
hdmi_in.start()
hdmi_out.start()

print ('3')

#将输入直接拉给输出（待测试：是否有了新的指定输出这条语句就会自动失效/此条语句是否可有可无）
#hdmi_in.tie(hdmi_out)

print ('4') 
state = 'run'  #工作模式选择。test为测试模式，跑一段就停下了。可以反复使用。run为演示模式，使用没有clean的while循环
# 调用`readflrame`函数将输入帧和输出帧存入缓存，并显示每秒帧数（存疑：numframes哪里来的，for后面为啥是个下划线（明白了，_是合法的变量名，
# 仅作为临时变量），for循环的作用（直观来看是将600帧图片（600个像素？）从readframe读到writeframe中））
if (state == 'test'):  
    print ('5') 
    numframes = 3000
    # 读取时间 以便输出fps---------（经测试 while循环无法上版）
    start = time.time()

    for _ in range(numframes):
        f = hdmi_in.readframe()
        hdmi_out.writeframe(f)

    end = time.time()
    print("Frames per second:  " + str(numframes / (end - start)))

elif (state == 'run'):
    print ('6') 
    while True: 
        f = hdmi_in.readframe()
        hdmi_out.writeframe(f)
        if base.buttons[0].read():
            print ('you said stop')
            break
        else:
            pass
#17s左右能到这里 6000的话30秒左右 可以持续相当久（比想象中久得多 但是画面有点问题（右边有一条跑左边去了）/又测试一次又正常了）
#使用while循环后快得多

#是不是因为没有clean 导致每次上电只能跑两趟
hdmi_out.close()
hdmi_in.close()
print ('7') 
  

# 实验一测试改动记录
1.测试
hdmi_out.close()
hdmi_in.close()
语句的影响
测试结果：不加的话基本跑两边就得重启板子，不然什么都加载不上去。添加此条语句后反复测试均没问题

2.测试
hdmi_in.tie(hdmi_out)
语句的影响
测试结果：注释掉与否未见显著影响

3.测试
添加测试1内容后，是否可以使用while循环
测试结果：竟然可以了！！！

4.测试
在测试3基础上实验视频流持续时间
测试结果：视频流起码持续了五分钟。未见异常

5.测试
在测试3结果上测试是否能重复加载
测试结果：不可以

6.改动
尝试调用键盘监听命令以clean接口
结果：很奇怪 似乎重启之后都无法加载了
结果2：多次重启后发现，板子上没有keyboard库 联网失败
解决方法（未实践）：调用板子上摁键做while的break

7.改动
在改动过6的基础上增加了工作模式选择。test为测试模式，跑一段就停下了。可以反复使用。run为演示模式，使用没有clean的while循环
改动结果：多次测试未见异常

8.改动：
增加了改动6提出的按键退出部分
运行正常


In [ ]:
#测试二：带opencv处理过程的输入输出
#测试1：简单的输入连接输出
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from pynq import PL
import cv2
import numpy as np
import time

print ('1') #6s
base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

print ('2') 

# 设置输入输出格式（hdmi_in设置为默认，hdmi_out设置为和in相同的格式）
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)

# 调用函数，开始hdmi输入输出
hdmi_in.start()
hdmi_out.start()

print ('3')

grayscale = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width), # 多个基于ny的空数组 使用ny效率高 (输入转灰度后缓存)
                       dtype=np.uint8)
result = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width), #输出缓存
                    dtype=np.uint8)
print ('4')
state = 'run' 
if (state == 'test'):
    numframes = 300
    star = time.time()
    for _ in range(numframes):
        inframe = hdmi_in.readframe()  # 将 hdmi_in.readframe() 函数的返回值赋给inframe，估计是一帧图像
        cv2.cvtColor(inframe,cv2.COLOR_BGR2GRAY,dst=grayscale) # 此函数为色彩空间转换，括号内为“原图像，转换类型，目标图像” 此处应该是BGR转灰度
        inframe.freebuffer() # 猜测是清空readframe寄存器
        cv2.Laplacian(grayscale, cv2.CV_8U, dst=result) #运行Laplacian算子 输出放入result数组

        outframe = hdmi_out.newframe() #猜测是定义一个新的可输出空白帧
        cv2.cvtColor(result, cv2.COLOR_GRAY2BGR,dst=outframe) #将result中存的缓存转换回BRG空间，然后输出
        hdmi_out.writeframe(outframe) #然后输出

    end = time.time()
    print("Frames per second:  " + str(numframes / (end - start)))
elif (state == 'run'):
    while True:
        inframe = hdmi_in.readframe()  # 将 hdmi_in.readframe() 函数的返回值赋给inframe，估计是一帧图像
        cv2.cvtColor(inframe,cv2.COLOR_BGR2GRAY,dst=grayscale) # 此函数为色彩空间转换，括号内为“原图像，转换类型，目标图像” 此处应该是BGR转灰度
        inframe.freebuffer() # 猜测是清空readframe寄存器
        cv2.Laplacian(grayscale, cv2.CV_8U, dst=result) #运行Laplacian算子 输出放入result数组

        outframe = hdmi_out.newframe() #猜测是定义一个新的可输出空白帧
        cv2.cvtColor(result, cv2.COLOR_GRAY2BGR,dst=outframe) #将result中存的缓存转换回BRG空间，然后输出
        hdmi_out.writeframe(outframe) #然后输出
        if base.buttons[0].read():
            print ('you said stop')
            break
        else:
            pass

hdmi_out.close()
hdmi_in.close()
print ('7') 

    

# 实验二测试改动记录
1.测试
一次成功。这个处理速度非常慢，300已经能刷好久了
问题：刷新率计算有问题。出来结果0.8，实际大概目测在3~5左右。可以300处以计时的方式计算出来

2.改动
修复了测试1测试出来的问题（但未成功）。理解了str(numframes / (end - start)的含义：总处理时间处以总处理帧数就是每一帧的处理时间，倒数就是fps
但是不知道为啥算出来fps0.4几，与目测结果不符

3.改动
增添状态部分，和实验一相同

4。改动
增添while循环的摁键中断

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from pynq import PL
import cv2
import numpy as np
import time

print ('所有库文件improt完成…') 
base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

print ('base overlay 及hdmi输入输出overlay实例化完毕…') 
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)

hdmi_in.start()
hdmi_out.start()

print ('视频输入格式捕获完成，输出格式设置完成 hdmi输入输出函数开启…')
grayscale = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width), # 多个基于ny的空数组 使用ny效率高 (输入转灰度后缓存)
                       dtype=np.uint8)
result = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width), #输出缓存
                    dtype=np.uint8)
result_reg = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width), #输出缓存
                    dtype=np.uint8)
firstframe = None
frameDelta = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width), # 差分结果缓存 其大小似乎还是原图
                    dtype=np.uint8)
thresh = np.ndarray(shape=(hdmi_in.mode.height, hdmi_in.mode.width), # 固定阈值二值化结果缓存，并且为膨胀后缓存
                    dtype=np.uint8)


numframes = 0
start = time.time()
while True: 
    
    if base.buttons[1].read():
        print ('检测到按键信号 系统停止工作')
        break

    numframes += 1
    inframe = hdmi_in.readframe()  # 将 hdmi_in.readframe() 函数的返回值赋给inframe，是一帧图像
    cv2.cvtColor(inframe,cv2.COLOR_BGR2GRAY,dst=grayscale) # 色彩空间转换，括号内为“原图像，转换类型，目标图像” 此处是BGR转灰度
    grayscale = cv2.dilate(grayscale, None, iterations=0)
    if firstframe is None:  # 如果没有上一张，则以当前帧作为评价标准，并跳出此次循环
        firstframe=grayscale
        print ('初始判定帧捕获完成 开始追踪工作…')
        continue
        
    frameDelta = cv2.absdiff(firstframe,grayscale) # 上一帧与当前帧做差分 返回值为差分结果
    thresh = cv2.threshold(frameDelta, 10, 255, cv2.THRESH_BINARY)[1] # 固定阈值二值化，括号内为（输入。阈值，超过阈值的赋值，二值化操作类型）
    
    thresh,contours,hierarch=cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) #获取二值化图像边缘 参考https://blog.csdn.net/xiaoxifei/article/details/82854068
    for i in range(len(contours)):        
        area = cv2.contourArea(contours[i])        
        if area < 100:    #此处10 为面积阈值            
            cv2.drawContours(thresh,[contours[i]],0,0,-1) #参数：第一个为待处理图像，第二个参量是将要处理的孤立区域轮廓Vector，第三个参量是表示轮廓的坐标，这里为0表示contours的第一个，第四个参量表示填充的数值

    x,y,w,h=cv2.boundingRect(thresh) #准备画框，括号内参数必须是一二值化图像，返回值就是把图像中“内容”框出来的起始点以及长宽。
    result=cv2.rectangle(inframe,(x,y),(x+w,y+h),(0,0,255),3) #在原图上画出来 放在result中
    inframe.freebuffer()
    
    outframe = hdmi_out.newframe() #定义一个新的可输出空白帧
    outframe[:]=result
    hdmi_out.writeframe(outframe) #输出
    outframe.freebuffer()

hdmi_out.close()
hdmi_in.close()
print ('系统工作结束 hdmi输入输出函数关闭')        
print("测试数据为：   " + "x="+str(x) + ", y="+ str(y) + ", w= "+ str(w) + ", h="+ str(h))
end = time.time()
print("fps为:  " + str(numframes / (end - start)))
print("每帧图像处理时间为:  " + str((end - start)/numframes))



# 测试或改动记录

1. 测试
    尝试直接注释掉高斯滤波模块
    测试结果：不显示图像
    
2. 异常
    退出幻灯片后板子会卡死。经测试，单纯的长时间使用不会导致上述问题，推测推出幻灯片会导致某种输出视频流的变化。
    处理方式：不要在连着板子的时候退出幻灯片。

3. 测试
    查看差分结果
    测试结果：边缘有异常多的噪声

4. 测试
    查看注释掉高斯滤波之后的差分结果
    测试结果：出现异常：没有输出了
    ？？？？
    加上高斯滤波之后 效果很好
    发现原因：cv2差分函数似乎需要某个cv2函数作为前置。他无法处理未经cv2函数处理过的图像。在这里我把一个侵蚀换成了之前的高斯滤波 效果很好

5. 测试
    在4的基础上 增加侵蚀参数，尝试去噪
    测试结果：失败，意识到噪声无法通过侵蚀处理掉
    咋还越侵蚀越亮。。
    我咋感觉我给他膨胀了呢
    换了个函数 测试一下
    
6. 直接取消第一步的处理就解决了
    这第一步是图啥啊。。
    
    
7. 为了预防可能出现的噪声问题，增添了一步处理，旨在删除二值化后图像中的小区域目标


8. 画面切换后噪声还是很多。尝试解决。
    我需要连接摄像头调试
    

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from pynq import PL
import cv2
import numpy as np
import time

print ('所有库文件improt完成…') 
base = BaseOverlay("base.bit")
Mode = VideoMode(640,480,24) 
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

# camera (input) configuration
frame_in_w = 640
frame_in_h = 480

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("capture device is open: " + str(videoIn.isOpened()))


print ('视频输入格式捕获完成，输出格式设置完成 hdmi输入输出函数开启…')
frame_vga = np.ndarray(shape=(frame_in_h, frame_in_w), # 输入缓存
                       dtype=np.uint8)
grayscale = np.ndarray(shape=(frame_in_h, frame_in_w), # 多个基于ny的空数组 使用ny效率高 (输入转灰度后缓存)
                       dtype=np.uint8)
result = np.ndarray(shape=(frame_in_h, frame_in_w), #输出缓存
                    dtype=np.uint8)
firstframe = None
frameDelta = np.ndarray(shape=(frame_in_h, frame_in_w), # 差分结果缓存 其大小似乎还是原图
                    dtype=np.uint8)
thresh = np.ndarray(shape=(frame_in_h, frame_in_w), # 固定阈值二值化结果缓存，并且为膨胀后缓存
                    dtype=np.uint8)

ret, frame_vga = videoIn.read()
numframes = 0
start = time.time()
while True: 
    
    if base.buttons[1].read():
        print ('检测到按键信号 系统停止工作')
        break

    numframes += 1
    cv2.cvtColor(frame_vga,cv2.COLOR_BGR2GRAY,dst=grayscale) # 色彩空间转换，括号内为“原图像，转换类型，目标图像” 此处是BGR转灰度
    grayscale=cv2.GaussianBlur(grayscale,(21,21),0) # 高斯滤波去噪 避免大量视频噪声干扰差分结果

    if firstframe is None:  # 如果没有上一张，则以当前帧作为评价标准，并跳出此次循环
        firstframe=grayscale
        print ('初始判定帧捕获完成 开始追踪工作…')
        continue
    
    winSize = (32, 32)      # winSize = (64,64)
    blockSize = (8,8)       # blockSize = (16,16)
    blockStride = (4,4)
    cellSize = (4,4)
    nbins = 9
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins)
 
    winStride = (4,4)
    padding = (4,4)
    locations = ((10,20),(30,30),(50,50),(70,70),(90,90),(110,110),(130,130),(150,150),(170,170),(190,190))
    hist = hog.compute(firstframe,winStride,padding,locations)
    print(hist)

    #outframe = hdmi_out.newframe() #定义一个新的可输出空白帧
    #outframe[:] = result
    #hdmi_out.writeframe(outframe) #输出
    #outframe.freebuffer()

hdmi_out.close()
print ('系统工作结束 hdmi输入输出函数关闭')        
#print("测试数据为：   " + "x="+str(x) + ", y="+ str(y) + ", w= "+ str(w) + ", h="+ str(h))
end = time.time()
print("fps为:  " + str(numframes / (end - start)))
print("每帧图像处理时间为:  " + str((end - start)/numframes))

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
from pynq import PL
import cv2
import numpy as np
import time

print ('所有库文件improt完成…') 
base = BaseOverlay("base.bit")
Mode = VideoMode(640,480,24) 
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

frame_in_w = 640
frame_in_h = 480

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("摄像头是否工作正常（True/fales）: " + str(videoIn.isOpened()))

#numpy群组声明
grayscale = np.ndarray(shape=(frame_in_h, frame_in_w), 
                       dtype=np.uint8)
result = np.ndarray(shape=(frame_in_h, frame_in_w), 
                    dtype=np.uint8)
firstframe = None
frameDelta = np.ndarray(shape=(frame_in_h, frame_in_w), 
                    dtype=np.uint8)
thresh = np.ndarray(shape=(frame_in_h, frame_in_w), 
                    dtype=np.uint8)

numframes = 0
start = time.time()
while True: 
    
    if base.buttons[1].read():
        print ('检测到按键信号 系统停止工作')
        break

    numframes += 1
    ret,frame_vga = videoIn.read()
    cv2.cvtColor(frame_vga,cv2.COLOR_BGR2GRAY,dst=grayscale) 

    if firstframe is None:  
        firstframe=grayscale
        print ('初始判定帧捕获完成 开始追踪工作…')
        continue    
    frameDelta = cv2.absdiff(firstframe,grayscale) 
    #grayscale=cv2.GaussianBlur(grayscale,(21,21),0) 
    grayscale = cv2.dilate(grayscale, None, iterations=0)
    thresh = cv2.threshold(frameDelta, 75, 255, cv2.THRESH_BINARY)[1] 
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2) 
    x,y,w,h=cv2.boundingRect(thresh) 
    result=cv2.rectangle(frame_vga,(x,y),(x+w,y+h),(0,0,255),3) 
    
    outframe = hdmi_out.newframe() 
    outframe[:] = result
    hdmi_out.writeframe(outframe) 
    outframe.freebuffer()

print ('test')  
hdmi_out.close()
videoIn.release()
print ('系统工作结束 hdmi输入输出函数关闭')        
print("测试数据为：   " + "x="+str(x) + ", y="+ str(y) + ", w= "+ str(w) + ", h="+ str(h))
end = time.time()
print("fps为:  " + str(numframes / (end - start)))
print("每帧图像处理时间为:  " + str((end - start)/numframes))